In [2]:
import pandas as pd
import re


In [3]:
zebraLogicBench = pd.read_parquet("hf://datasets/allenai/ZebraLogicBench/grid_mode/test-00000-of-00001.parquet")

In [4]:
zebraLogicBench[0:2]

,id,size,puzzle,solution,created_at
0,lgp-test-5x6-16,5*6,"There are 5 houses, numbered 1 to 5 from left ...","{'header': ['House', 'Name', 'Nationality', 'B...",2024-07-11T01:22:10.734298
1,lgp-test-4x4-27,4*4,"There are 4 houses, numbered 1 to 4 from left ...","{'header': ['House', 'Name', 'Occupation', 'Bo...",2024-07-11T01:22:10.732670


In [5]:
puzzle1 = zebraLogicBench['puzzle'][0]
print(puzzle1)

There are 5 houses, numbered 1 to 5 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Peter`, `Alice`, `Bob`, `Eric`, `Arnold`
 - The people are of nationalities: `norwegian`, `german`, `dane`, `brit`, `swede`
 - People have unique favorite book genres: `fantasy`, `biography`, `romance`, `mystery`, `science fiction`
 - Everyone has something unique for lunch: `stir fry`, `grilled cheese`, `pizza`, `spaghetti`, `stew`
 - Each person has a favorite color: `red`, `green`, `blue`, `yellow`, `white`
 - The people keep unique animals: `bird`, `dog`, `cat`, `horse`, `fish`

## Clues:
1. The person who loves fantasy books is the Norwegian.
2. The cat lover and the person who loves biography books are next to each other.
3. The German is Bob.
4. The person who loves yellow is Bob.
5. The person whose favorite color is green is Peter.
6. 

In [6]:
# result = re.findall(r'There are (\d+) houses, numbered 1 to \d+ from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:', puzzle1)
result = re.findall(r'There are (\d+) houses, numbered 1 to \d+ from left to right', puzzle1)

if len(result) != 1:
    print("Invlaid count")

house_count = int(result[0])

house_count

5

In [7]:

# result = re.findall(r' - (.*?): (`(.*?)`)(?:, (`(.*?)`))*', puzzle1)

x = []
for i in range(0, house_count):
    x.append('`(.*?)`')

var_parser = ", ".join(x)

variables = re.findall(r' - (.*?): ' + var_parser, puzzle1)
variables


[('Each person has a unique name', 'Peter', 'Alice', 'Bob', 'Eric', 'Arnold'),
 ('The people are of nationalities',
  'norwegian',
  'german',
  'dane',
  'brit',
  'swede'),
 ('People have unique favorite book genres',
  'fantasy',
  'biography',
  'romance',
  'mystery',
  'science fiction'),
 ('Everyone has something unique for lunch',
  'stir fry',
  'grilled cheese',
  'pizza',
  'spaghetti',
  'stew'),
 ('Each person has a favorite color',
  'red',
  'green',
  'blue',
  'yellow',
  'white'),
 ('The people keep unique animals', 'bird', 'dog', 'cat', 'horse', 'fish')]

In [8]:
# 1. The person who loves fantasy books is the Norwegian.
clues = re.findall(r'\d+. (.*?)\.', puzzle1)


clues

['The person who loves fantasy books is the Norwegian',
 'The cat lover and the person who loves biography books are next to each other',
 'The German is Bob',
 'The person who loves yellow is Bob',
 'The person whose favorite color is green is Peter',
 'There is one house between the Dane and the person who is a pizza lover',
 'The person who loves blue is somewhere to the left of the Dane',
 'The person who loves eating grilled cheese is somewhere to the left of the Norwegian',
 'The person who loves the spaghetti eater is Peter',
 'The person who keeps horses is Alice',
 'The fish enthusiast is directly left of the person who loves science fiction books',
 'There is one house between the Norwegian and Arnold',
 'The person who loves romance books is the British person',
 'There are two houses between the Norwegian and Alice',
 'The bird keeper is the person whose favorite color is red',
 'The dog owner is directly left of the fish enthusiast',
 'The person who loves the stew is the 

In [169]:
def analyze_puzzle_text(text):
    # House count
    result = re.findall(r'There are (\d+) houses, numbered 1 to \d+ from left to right', text)

    if len(result) != 1:
        raise Exception("Invalid house count")

    house_count = int(result[0])
    
    # Variables
    x = []
    for i in range(0, house_count):
        x.append('`(.*?)`')

    var_parser = ", ".join(x)
    variables = re.findall(r' - (.*?): ' + var_parser, text)
    
    variables2 = []
    for group in variables:
        group_name = group[0]
        l = []
        l.append(group_name)
        for v in group[1:]:
            if ("child" in group_name):
                l.append(f'child is named {v}')
                l.append(f'mother of {v}')
                
            elif "month" in group_name:
                if v == "jan": 
                    l.append("january")
                else:
                    l.append(v)
                
            elif "hip hop" == v:
                l.append("hip-hop")
                
            elif "swede" == v:
                l.append("swedish")
            
            elif "ford f150" == v:
                l.append("Ford F-150")
            
            elif "cat" == v: 
                l.append(" cat") # prevent match with vacation
                
            else:
                if v.endswith('ing'):
                    v = v[:-3]
                elif v.endswith('s'):
                    v = v[:-1]
                l.append(v)
        
        variables2.append(l)
            
    
    # clues
    clues = re.findall(r'\d+. (.*?)\.', text)
    
    return (house_count, variables2, clues)

In [185]:
def check_for_clue(var1, var2, regex, text):
    
    sa = regex.replace("%1", var1).replace("%2", var2)
    if re.search(sa, text, re.IGNORECASE):
        return True
    
    sb = regex.replace("%1", var2).replace("%2", var1)
    if re.search(sb, text, re.IGNORECASE):
        return True
    
    return False

def check_for_single_clue(var1, regex, text):
    
    sa = regex.replace("%1", var1)
    if re.search(sa, text, re.IGNORECASE):
        return True
    
    return False

def analyze_clue(vars, clue):
    
    if len(vars) == 1:
        if check_for_single_clue(vars[0], "%1(.*?) first house", clue):
            return "is1"
        if check_for_single_clue(vars[0], "%1(.*?) second house", clue):
            return "is2"
        if check_for_single_clue(vars[0], "%1(.*?) third house", clue):
            return "is3"
        if check_for_single_clue(vars[0], "%1(.*?) fourth house", clue):
            return "is4"
        if check_for_single_clue(vars[0], "%1(.*?) fifth house", clue):
            return "is5"
        if check_for_single_clue(vars[0], "%1(.*?) sixth house", clue):
            return "is6"
    
    if len(vars) == 2:
        if check_for_clue(vars[0], vars[1], "one house between(.*?)%1(.*?)%2", clue):
            return "oneBetween"
        if check_for_clue(vars[0], vars[1], "two houses between(.*?)%1(.*?)%2", clue):
            return "twoBetween"
        if check_for_clue(vars[0], vars[1], "%1(.*?)%2(.*?)next to each other", clue):
            return "nextTo"
        if check_for_clue(vars[0], vars[1], "%1(.*?)directly left of(.*?)%2", clue):
            return "dLeftOf"
        if check_for_clue(vars[0], vars[1], "%1(.*?)left of(.*?)%2", clue):
            return "leftOf"
        if check_for_clue(vars[0], vars[1], "%1(.*?)directly right of(.*?)%2", clue):
            return "dRightOf"
        if check_for_clue(vars[0], vars[1], "%1(.*?)right of(.*?)%2", clue):
            return "rightOf"
        # if check_for_clue(vars[0], vars[1], "%1(.*?)is (the |a )?%2", clue):
        if check_for_clue(vars[0], vars[1], "%1(.*?)is(.*?)%2", clue):
            return "equal"

    return None


def analyze_clues(puzzle):

    all_variables = []
    for group in puzzle[1]:
        for var in group:
            all_variables.append(var)
    all_variables = sorted(all_variables, key=len, reverse=True)
    
    clues = []
    for c in puzzle[2]:
        vars = []
                   
        # sometimes a variable contains another variables value (eg: 'child of alice' and 'alice')
        # all variables are sorted by length, each match is the removed from the clue text.
        # This ensures that none of the shorter variables can match a part from a longer variable
        test_clue = c
        for var in all_variables:
            if re.search(var, test_clue, re.IGNORECASE):
                vars.append(var)
                test_clue = test_clue.replace(var, "")
        
        
        func = analyze_clue(vars, c)
        
        clues.append( (c, vars, func) )

    return (puzzle[0], puzzle[1], clues)
    

puzzle = zebraLogicBench['puzzle'][3]
p1 = analyze_puzzle_text(puzzle)
_, _, clues = analyze_clues(p1)

for c in clues:
    if c[2] is None:
        print(c[1], c[0])
    # display((c[2], c[0]))
    # print(c[2], c[0])

print(p1[1])


[['Each person has a unique name', 'Bob', 'Arnold', 'Carol', 'Alice', 'Peter', 'Eric'], ["The mothers' names in different houses are unique", 'Sarah', 'Janelle', 'Aniya', 'Kailyn', 'Holly', 'Penny'], ['Each mother is accompanied by their child', 'child is named Fred', 'mother of Fred', 'child is named Samantha', 'mother of Samantha', 'child is named Bella', 'mother of Bella', 'child is named Meredith', 'mother of Meredith', 'child is named Alice', 'mother of Alice', 'child is named Timothy', 'mother of Timothy'], ['Each person prefers a unique type of vacation', 'city', 'mountain', 'camp', 'beach', 'cruise', 'cultural'], ['People have unique favorite book genres', 'romance', 'mystery', 'historical fiction', 'science fiction', 'biography', 'fantasy']]


In [177]:
re.search("tea", "The person who is a teacher is directly left of the tea drinker")

<re.Match object; span=(20, 23), match='tea'>

In [186]:

count_total = 0
count_broken = 0

index = 0
first_broken = -1
broken = []

for puzzle in zebraLogicBench['puzzle']:
    p1 = analyze_puzzle_text(puzzle)
    _, _, clues = analyze_clues(p1)
    
    unsuccessfull = False
    for c in clues:
        if c[2] is None:
            unsuccessfull = True
    
    if unsuccessfull:
        count_broken = count_broken + 1 
        
        if first_broken == -1:
            first_broken = index
        broken.append(index)
        
    count_total = count_total + 1
    
    index = index + 1
    
display(broken)
print(count_broken, count_total, round(count_broken / count_total * 100), '%')

[18, 81, 175, 193, 236, 289, 420, 459, 800, 830, 850, 922, 969, 974, 999]

15 1000 2 %
